# MFPT Fault Dataset Manipulation

<img src="https://raw.githubusercontent.com/cherrerab/deeplearningfallas/master/workshop_01/bin/mfpt_logo.png" width="400">

La Machinery Failure Prevention Technology es una organización orientada a la integración y al intercambio de información técnica entre distintas comunidades científicas y de ingeniería, con el fin de avanzar en el estudio de los mecánismos de falla.

https://www.mfpt.org/

En este workshop trabajaremos con su [Bearing Fault Dataset](https://www.mfpt.org/fault-data-sets/), el cual contiene mediciones experimentales de vibración de rodamientos de bola bajo distintos estados de daño. Fallas localizadas en este tipo de componentes pueden ocurrir en la ranura del anillo exterior, en la ranura del anillo interior, o bien, en alguno de los elementos rodantes del rodamiento. Cuando los elementos rodantes pasan o impactan con alguno de estos defectos, se producen vibraciones de alta frecuencia que pueden ser registradas, por ejemplo, mediante un acelerómetro o un transductor.

<img src="https://raw.githubusercontent.com/cherrerab/deeplearningfallas/master/workshop_01/bin/bearing_example.png" width="500">

El propósito de este dataset es propiciar la investigación y el desarrollo de modelos capaces de detectar e identificar el tipo de falla en un rodamiento, en base a las señales medidas y registradas por un transductor de vibración.

El dataset que se usará en este workshop consiste en tres señales de aceleración (g) registradas durante tres segundos (2.99 seg) a un sample rate de 4882.8 Hz sobre rodamientos NICE® a 1500 rpm. Estos registros se corresponden con tres estados de salud: `healthy`, `outer race fault`, `inner race fault`.

El dataset se encuentra en el archivo `MFPT_raw_dst.csv` contenido en el github del curso.

In [ ]:
!git clone https://github.com/cherrerab/deeplearningfallas.git
%cd /content/deeplearningfallas


---
# Exploración Inicial

Comenzemos por importar el dataset a un `pandas.DataFrame` para observar su contenido. El archivo se encuentra en el path `\content\deeplearningfallas\workshop_01\MFPT_raw_dst.csv`.

In [ ]:
import numpy as np
import pandas as pd

# ubicación del dataset
dst_path = 'workshop_01//MFPT_raw_dst.csv'

# importar a un pd.DataFrame


# imprimir head del dataset (primeras 5 filas)


# imprimir cantidad de datos contenidos
n_data = 
print( '\nsamples: ', n_data )

# imprimir tiempo total registrado
t_total = 
print( '\ntime total: {:1.7f} seg'.format(t_total) )

# imprimir time step y sample rate de la señal
t_step = 
print( '\ntime step: {:1.7f} seg'.format(t_step) )

sample_rate = 
print( '\nsample rate: {:1.1f} Hz'.format(sample_rate) )


Como se puede ver, el dataset cuenta con 14,649 samples de aceleración registrados durante 3 segundos de medición. Por supuesto, el dataset cuenta con datos de tres estados distintos de salud.

Ahora, parte importante de una buena exploración de datos consiste en visualizarlos, esto nos permitirá manejar una mejor intuición del comportamiento y los patrones que estos presenten, así como sus posibles anomalías. Para esto, usaremos principalmente `matplotlib`, la cual es una librería orientada y especializada en la creación de gráficos y visualizaciones en python.

https://matplotlib.org/

De este modo, lo primero que haremos será plotear los datos de aceleración de cada uno de los estados de aceleración en el tiempo.

In [ ]:
import matplotlib.pyplot as plt

# gráfico de estado de salud Healthy
plt.figure( figsize=(15, 4) )
plt.title('Healthy')
plt.xlabel('Time (seg)'); plt.ylabel('Acceleration (g)')

# gráfico de estado de salud Outer Race


# gráfico de estado de salud Inner Race



Se puede observar que existe un diferencia evidente entre las tres señales, donde la magnitud de la aceleración es lo que cambia de forma más notoria. Podemos cuantificar esta diferencia fácilmente mediante `np.max` y `np.abs`.

In [ ]:
# obtendremos la magnitud de la señal en cada sample mediante np.abs
db_abs = 

# de este modo, podremos obtener la amplitud máxima de cada señal
# mediante np.max o pd.DataFrame.max
db_max = 
print( db_max )

## Spectrograms

Ahora, dado que los datos consisten en señales de vibración, puede ser útil para la exploración analizar los espectrogramas de cada una. Por supuesto, en python practicamente ya está todo implementado y solo basta buscar la librería adecuada. Una rápida búsqueda por google nos lleva a `scipy.signal.spectrogram`.

<img src="https://raw.githubusercontent.com/cherrerab/deeplearningfallas/master/workshop_01/bin/scipy_badge.png" width="350">

Dado que debemos crear tres espectrogramas, uno para cada serie temporal, crearemos la función `MFPT_spectrogram` que nos permitirá repetir el proceso.

In [ ]:
from scipy import signal

def MFPT_spectrogram(data, sr=1.0, nperwd=256):
  """
  -> None

  compute and plot the spectrogram of the time series.
  spectrograms can be used as a way of visualizing the change of a
  nonstationary signal’s frequency content over time.

  :param pd.Series data:
    time series of measurement values.
  :param float sr:
    sample rate of the time series data. defaults to 1.0.
  :param int nperwd:
    length of samples of each time window. defaults to 256.

  :returns:
    None
  """

  # obtener espectrograma
  freqs, times, spectrogram = signal.spectrogram(data, fs=sr, nperseg=nperwd)

  # crear meshgrid para plot
  X, Y = np.meshgrid(times, freqs)

  # configurar plot
  plt.figure( figsize=(15, 4) )
  plt.title( data.name )
  plt.xlabel('Time (seg)'); plt.ylabel('Frecueny (Hz)')

  # plot
  plt.contourf( X, Y, spectrogram, levels=256, cmap='magma')
  plt.colorbar()

  return None

# obtener espectrograma Healthy


# obtener espectrograma Outer Race


# obtener espectrograma Inner Race




Si bien en el caso `healthy` el espectrograma indica que la señal consiste de básicamente ruido/noise, tanto en el caso `outer race fault` como en el `inner race fault` se distinguen patrones que indican la presencia de anomalías en la señal.



---


# Training Data
Eventualmente, lo que queremos hacer con este dataset es generar un modelo de deep learning que, a partir de una muestra o sample de una señal de vibración, sea capaz de identificar el estado de salud o tipo de falla que presente el componente. Es decir un modelo de clasificación.

Ahora bien, para generar este modelo de clasificación, es necesario contar con una serie de muestras para su entrenamiento. Durante este entrenamiento, el modelo observará estas muestras e iterativamente irá ajustando sus pesos de tal manera de reducir su error de clasificación. Es de esta manera que el modelo logra aprender de los datos.


## Time Windows

En general, mientras más muestras se posea para el entrenamiento, mejor. No obstante, nunca contaremos con datasets infinitos por lo que debemos ser precavidos y eficientes respecto a como distribuimos nuestros datos. En este caso particular, contamos con tres señales finitas a partir de las cuales tendremos que extraer nuestras muestras. De este modo, nuestras muestras consistirán en ventanas de datos o tiempo, similar a como los espectrogramas anteriores procesan la información.

A continuación, definiremos la función `get_time_windows` que nos permita extraer las ventanas de tiempo de una serie de datos, en función de un tamaño de ventana `nperwd` definido.

In [ ]:
def get_time_windows(data, nperwd):
  """
  -> np.array

  generates a numpy array of time windows, of length nperwd, extracted
  from data.

  :param pd.Series data:
    time series of measurement values.
  :param int nperwd:
    length of samples of each time window.
  :param int overlap:
    length of overlap between time windows.

  :returns:
    a numpy array of size (n_windows, nperwd).
  """

  # obtener np.array de la serie de datos
  

  # determinar cantidad de ventanas a generar
  

  # generar time windows
  

  return out

# generar ventanas a partir de los datos Inner Race
X_IR = get_time_windows( db['Inner Race'], nperwd=128 )

print('time windows generated: {:d}'.format(X_IR.shape[0]))

# plot muestra de ventana
x = 

plt.figure( figsize=(15, 4) )
plt.title('Inner Race TW 12')
plt.xlabel('Data point'); plt.ylabel('Acceleration (g)')
plt.plot( x )

Una pregunta que nos podríamos hacer ahora, es respecto a de qué tamaño deben ser estas ventanas. La intuición nos indica que mientras más ventanas generemos, mejor será nuestro entrenamiento. No obstante, si definimos `nperwd` muy pequeño es posible que nuestras ventanas no contengan suficiente información.

<img src="https://raw.githubusercontent.com/cherrerab/deeplearningfallas/master/workshop_01/bin/nperwd_panik.png" height="300">

De esta manera, debemos encontrar un `nperwd` que logre un buen balance entre la cantidad de ventanas o samples generados, y la cantidad de información que estos contengan. Por supuesto, una forma de hacer esto es mediante prueba y error, pero también es posible explorar métodos un tanto más sistemáticos.

Dado que este es el primer workshop del curso, nos limitaremos a definir el `nperwd` que nos parezca adecuado, por ejemplo, `48`. Ahora crearemos el `training set` para nuestro modelo, utilizando las tres señales de vibración.

In [ ]:
# definir tamaño de ventanas
nperwd = 48

# generar ventanas a partir de los datos Healthy
X_HB = 

# generar ventanas a partir de los datos Outer Race
X_OR = 

# generar ventanas a partir de los datos Inner Race
X_IR = 

# finalmente, concatenar estas ventanas mediante np.vstack
X_raw = 

print('number of samples: ', X_raw.shape[0])
print('samples length: ', X_raw.shape[1])


Por supuesto, como nuestro modelo debe aprender a clasificar el estado de salud de nuetros `samples`, debemos crear un `np.array` `target` que contenga las etiquetas de nuestras muestras.

In [ ]:
# generar np.array labels con las etiquetas correspondientes
labels = 

# reshape a dimensiones compatibles con X_raw
Y = labels.reshape( (-1, 1) )


## Feature Extraction

Hasta este punto, salvo quizás por los espectrogramas, nos hemos limitado a utilizar únicamente los datos puros o `raw`. No obstante, en Machine Learning, es común preprocesar los datos antes de entrenar los modelos y no utilizar los datos en `raw`. El objetivo de esto es extraer `features` y valores que sean más efectivos a la hora del entrenamiento.

Para introducir esta etapa de preprocesamiento, definiremos la función `extract_features`. Esta función procesará las ventanas de datos generadas en la sección anterior, y calculará algunas `features` características de señales de vibración: `rms`, `peak`, `valley`, `peak2peak`.



In [ ]:
def extract_features(x):
  """
  -> pd.DataFrame

  compute signal features for each sample in the data x.

  :param np.array x:
    data of shape (n_samples, nperwd) containing de samples.

  :returns:
    pd.DataFrame of shape (n_samples, n_features) containing
    the extracted features.
  """
  
  # valor eficaz rms
  rms = 

  # peak
  peak = 

  # valley
  valley = 

  # peak2peak
  peak2peak = 

  # concatenar features mediante np.hstack
  out = 

  # generar pd.DataFrame
  db = pd.DataFrame(out, columns=['rms', 'peak', 'valley', 'peak2peak'])

  return db

# generar dataset de features a partir de X_raw
db_ft = extract_features(X_raw)
print( db_ft.head(n=5) )

# obtener X_features mediante np.array.values
X_features = 

Finalmente, podemos visualizar la distribución que presentan estos features en cada uno de los estados, para hacernos una idea de que tan útiles podrían resultar para el entrenamiento.

In [ ]:
# scatter plot RMS
plt.figure( figsize=(15, 4) )
plt.xlabel('Sample'); plt.ylabel('RMS')

plt.scatter(db_ft.index, db_ft['rms'],
            c=list(labels), s=10, cmap='winter')